# `05-websearch`

# Tavily Web Search
- LLM에게 웹 검색을 쥐어주자

In [1]:
%pip install -q langchain-tavily

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import  load_dotenv

load_dotenv()

In [2]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None,
    # country=None
)

In [ ]:
search_tool.invoke({'query': '오늘 호랑이띠 운세'})

{'query': '오늘 호랑이띠 운세',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.joongangenews.com/news/articleView.html?idxno=447303',
   'title': '[오늘의 운세] 띠별 운세, 호랑이띠의 9월 3일 운세가 전하는 ...',
   'content': '호랑이띠는 용기와 기백을 상징하며, 오늘은 도전의 기운이 강하게 흐른다. 자신의 뜻을 밀고 나가고 싶은 마음이 커지지만, 조율의 지혜가 필요하다.',
   'score': 0.9108078,
   'raw_content': None},
  {'url': 'https://ineuro.co.kr/',
   'title': '오늘의띠별운세 무료 : 오늘에운세 띠운세',
   'content': '호랑이띠(寅) 호랑이띠는 강한 자신감과 리더십을 지닌 인물이에요. 도전 정신이 강해 새로운 일에 겁 없이 뛰어들고, 분위기를 주도하는 힘이 있죠. 감정 표현이 솔직해',
   'score': 0.8991304,
   'raw_content': None},
  {'url': 'https://www.newsfreezone.co.kr/news/articleView.html?idxno=644650',
   'title': '[오늘의 띠별 운세] 9월 3일',
   'content': '사소한 일도 철저히 하라. 호랑이띠. 62년생: 건강을 소홀히 하면 낭패가 있다. 작은 증상도 살펴라. 74년생: 귀인의 도움으로 막힌 일이 풀린다. 86',
   'score': 0.8440111,
   'raw_content': None},
  {'url': 'https://freesaju.net/unjoa/12animal/ddi.php?d=%C8%A3%B6%FB%C0%CC',
   'title': '호랑이띠운세 오늘의 범띠 띠별운세',
   'content': '범

In [4]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

'강남 신세계백화점에서 점심 식사 추천으로는 타치바나, 플레이버즈, 베통 등의 맛집이 인기 있으며, 미쉐린 빕 구르망 셰프의 점심 한정 반상도 좋은 선택입니다. 다양한 맛집과 점심 메뉴가 있어 취향에 맞게 선택하시면 좋습니다.'

In [5]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

'강남 신세계 내에서는 다양하고 맛있는 점심 식사 옵션이 있어요. 대표적으로 신세계 백화점 내에 위치한 프리미엄 레스토랑이나 카페에서 고급스럽고 편안한 식사를 즐기실 수 있습니다. 또한, 신세계 주변에는 한식, 일식, 양식 등 다양한 음식점들도 많아 취향에 맞게 선택하시면 좋을 것 같아요.'

In [16]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime

today = datetime.today().strftime('%D')

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해야해 최대한 답을 잘 해보자. 오늘은 {today} 야'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출 때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[search_tool], verbose=True, memory=memory)

In [19]:
agent_executor.invoke({'input': '내 이름 알잖아, 이름 인터넷에 찾아봐'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '하츄핑'}`


{'query': '하츄핑', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://ko.wikipedia.org/wiki/%ED%95%98%EC%B8%84%ED%95%91', 'title': '하츄핑 - 위키백과, 우리 모두의 백과사전', 'content': '하츄핑(영어: Heartsping)은 대한민국의 마법소녀 애니메이션 《캐치! 티니핑》의 주인공으로, 매 시즌마다 주제에 맞는 명칭이 따라온다. 1기에는 일반 하츄핑, 2기에는 다이아 하츄핑, 3기에는 플로라 하츄핑, 4기에는 베리 하츄핑, 5기에는 스타 하츄핑이다.', 'score': 0.89567816, 'raw_content': None}, {'url': 'https://catchteenieping.fandom.com/ko/wiki/%ED%95%98%EC%B8%84%ED%95%91', 'title': '하츄핑 | 캐치! 티니핑 위키 | Fandom', 'content': '하츄핑은 핑크 다이아몬드(시즌 2)를 휘두르는 사랑의 로열 티니핑이다. 그녀는 항상 이모션 왕국의 공주, 로미와 함께 있다. 로미를 프린세스 하트로 변신시켜준다.', 'score': 0.8055255, 'raw_content': None}, {'url': 'https://ko.wikipedia.org/wiki/%EC%82%AC%EB%9E%91%EC%9D%98_%ED%95%98%EC%B8%84%ED%95%91', 'title': '사랑의 하츄핑 - 위키백과, 우리 모두의 백과사전', 'content': '《사랑의 하츄핑》(영어: Heartsping : Teenieping of Love)은 2024년 8월 7일에 개봉한 대한민국의 애니메이션 영화이다. 캐치! 티니핑 시

{'input': '내 이름 알잖아, 이름 인터넷에 찾아봐',
 'chat_history': [HumanMessage(content='나는 하츄핑이야', additional_kwargs={}, response_metadata={}),
  AIMessage(content='반가워, 하츄핑! 너에 대해 더 알고 싶거나 도움이 필요한 것이 있다면 알려줘. 어떻게 도와줄까?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='나는 어떻게 생겼어?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='나는 네가 어떻게 생긴지 직접 볼 수 없기 때문에, 네가 어떤 모습인지 구체적으로 설명해주면 좋아. 또는 네가 이미 알고 있는 모습에 대해 이야기해줄 수도 있고. 네가 어떤 모습인지 묘사해줄 수 있어?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='내 이름 알잖아, 이름 인터넷에 찾아봐', additional_kwargs={}, response_metadata={}),
  AIMessage(content='네 이름인 "하츄핑"은 대한민국의 애니메이션 "캐치! 티니핑"의 주인공 이름인 것 같아. 하츄핑은 사랑의 로열 티니핑으로, 배려심이 많고 다정한 성격의 캐릭터야. \n\n혹시 사람 이름이 아니고 캐릭터 이름이라면, 지금 내가 찾은 정보는 애니메이션 속 캐릭터에 관한 내용이야. 네가 어떤 모습인지 더 구체적으로 알려줄 수 있으면, 내가 더 도와줄 수 있어!', additional_kwargs={}, response_metadata={})],
 'output': '네 이름인 "하츄핑"은 대한민국의 애니메이션 "캐치! 티니핑"의 주인공 이름인 것 같아. 하츄핑은 사랑의 로열 티니핑으로, 배려심이 많고 다정한 성격의 캐릭터야. \n\n혹시 사람 이름이 아니고